# Actividad 3:
# Optimización Inteligente de Modelos Predictivos en Salud usando Técnicas Bayesianas

## Objetivo
Aplicar la Optimización Bayesiana para ajustar los hiperparámetros de un modelo de clasificación binaria (Random Forest) sobre un problema de salud pública, comparando dos enfoques populares: Scikit-Optimize (skopt) y Hyperopt, evaluando el rendimiento del modelo y la eficiencia de cada técnica.

**Dataset utilizado:**  
Cáncer de mama de Scikit-learn

---

### Estructura del Notebook:
1. Metodología.
2. Configuración inicial del notebook.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

---

### Flujo de trabajo
1. **Carga, preprocesado de datos y creación del modelo base:**
   - Carga datos, los preprocesa (escalado) y crea el modelo base.

2. **Aplicación de técnicas de optimización bayesiana:**
   - Optimización bayesiana usando BayesSearchCV (Scikit-Optimize).
   - Optimización bayesiana usando Hyperopt con validación cruzada.

3. **Evaluación comparativa:**
   - Métricas del modelo base, optimizado con BayesSearchCV e Hyperopt por separado y su comparación (F1-Score, tiempos).
   - Comparación de métricas de ambos métodos de optimización con distinto número de iteraciones y espacios de busqueda (F1-Score, tiempos).
   - Visualización de las comparaciones.

---

# 2. Configuración inicial del notebook
- Importación de librerias necesarias.
- Configuraciones necesarias para el correcto manejo de las salidas del código.

--- 

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from skopt import BayesSearchCV
from skopt.space import Integer
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# Configuración de pandas para mostrar todos los parámetros
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Funciones de preprocesamiento de datos.

- **`load_data()`** 
Carga el dataset de cáncer de mama, aplica escalado estándar y lo divide en conjuntos de entrenamiento y prueba.

- **`train_base_rf()`** 
Entrena un modelo RandomForestClassifier sin ajuste de hiperparámetros y devuelve métricas de evaluación.

In [ ]:
def load_and_prepare_data(data, test_size=0.3, random_state=42):
    """
    Carga y prepara el dataset de cáncer de mama.
    
    Realiza:
    - Carga del dataset
    - Escalado de características con StandardScaler
    - División estratificada train/test (70/30)
    
    Parámetros:
    test_size (float): Proporción para test (default 0.3)
    random_state (int): Semilla para reproducibilidad (default 42)
    
    Retorna:
    tuple: (X_train, X_test, y_train, y_test) arrays preprocesados
    """
    #data = load_breast_cancer()
    X = data.data
    y = data.target
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

def train_base_rf(X_train, y_train, X_test, y_test):
    """
    Entrena y evalúa un RandomForest sin ajuste de hiperparámetros.
    
    Parámetros:
    X_train, y_train: Datos de entrenamiento
    X_test, y_test: Datos de prueba
    
    Retorna:
    tuple: (reporte_clasificación, f1_score, tiempo_entrenamiento)
    """
    rf = RandomForestClassifier(random_state=42)
    start_time = time.time()
    rf.fit(X_train, y_train)
    train_time = time.time() - start_time
    y_pred = rf.predict(X_test)
    report = classification_report(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return report, f1, train_time

**Bloque 2:** Funciones de optimización bayesiana.

- **`bayes_opt_skopt()`** 
Realiza optimización bayesiana de hiperparámetros usando BayesSearchCV y retorna el mejor modelo y sus métricas.

- **`bayes_opt_hyperopt()`** 
Ejecuta optimización bayesiana usando Hyperopt y fmin con validación cruzada, entrenando luego el mejor modelo encontrado.

In [ ]:
def bayes_opt_skopt(X_train, y_train, X_test, y_test, n_iter=30):
    """
    Optimización bayesiana usando BayesSearchCV (Scikit-Optimize).
    
    Define espacio de búsqueda para:
    - n_estimators: [50, 500]
    - max_depth: [2, 30]
    - min_samples_split: [2, 30]
    
    Parámetros:
    X_train, y_train: Datos de entrenamiento
    X_test, y_test: Datos de prueba
    n_iter (int): Número de iteraciones de optimización
    
    Retorna:
    tuple: (mejores_parámetros, reporte_clasificación, f1_score, tiempo_total)
    """
    search_space = {
        'n_estimators': Integer(50, 500),
        'max_depth': Integer(2, 30),
        'min_samples_split': Integer(2, 30)
    }
    rf = RandomForestClassifier(random_state=42)
    opt = BayesSearchCV(
        rf,
        search_spaces=search_space,
        n_iter=n_iter,
        cv=3,
        scoring='f1',
        random_state=42,
        n_jobs=-1
    )
    start = time.time()
    opt.fit(X_train, y_train)
    elapsed = time.time() - start
    best_params = opt.best_params_
    y_pred = opt.predict(X_test)
    report = classification_report(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return best_params, report, f1, elapsed

def bayes_opt_hyperopt(X_train, y_train, X_test, y_test, max_evals=30):
    """
    Optimización bayesiana usando Hyperopt con validación cruzada.
    
    Define espacio de búsqueda con distribuciones uniformes discretas para:
    - n_estimators: [50, 500]
    - max_depth: [2, 30]
    - min_samples_split: [2, 30]
    
    Parámetros:
    X_train, y_train: Datos de entrenamiento
    X_test, y_test: Datos de prueba
    max_evals (int): Número máximo de evaluaciones
    
    Retorna:
    tuple: (mejores_parámetros, reporte_clasificación, f1_score, tiempo_total)
    """
    space = {
        'n_estimators': hp.quniform('n_estimators', 50, 500, 1),
        'max_depth': hp.quniform('max_depth', 2, 30, 1),
        'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
    }
    
    def objective(params):
        params_int = {
            'n_estimators': int(params['n_estimators']),
            'max_depth': int(params['max_depth']),
            'min_samples_split': int(params['min_samples_split']),
            'random_state': 42,
            'n_jobs': -1
        }
        clf = RandomForestClassifier(**params_int)
        
        # Usar validación cruzada para evitar sobreajuste
        score = cross_val_score(clf, X_train, y_train, cv=3, scoring='f1').mean()
        return {'loss': -score, 'status': STATUS_OK}
    
    trials = Trials()
    start = time.time()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.default_rng(42)
    )
    elapsed = time.time() - start
    
    best_params = {
        'n_estimators': int(best['n_estimators']),
        'max_depth': int(best['max_depth']),
        'min_samples_split': int(best['min_samples_split']),
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Entrenar modelo final con todos los datos de entrenamiento
    final_model = RandomForestClassifier(**best_params)
    final_model.fit(X_train, y_train)
    y_pred = final_model.predict(X_test)
    report = classification_report(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    return best_params, report, f1, elapsed

**Bloque 5:** Funciones de visualización.

- **`plot_comparison()`** 
Genera un gráfico comparativo de F1-Score y tiempos de ejecución entre el modelo base, BayesSearchCV y Hyperopt.

- **`plot_experiment_results()`** 
Crea cuatro gráficos que muestran el impacto de las iteraciones y los rangos en el F1-Score y el tiempo de entrenamiento.

- **`create_main_results_table()`** 
Construye DataFrames con los resultados clave (f1-score, tiempo y reporte) para el modelo base y los optimizados.

- **`create_experiment_tables()`** 
Genera tablas con los resultados de los experimentos de iteraciones y rangos, diferenciando entre métodos.

- **`display_formatted_tables()`** 
Muestra de forma organizada y con formato legible los resultados principales y experimentales en la consola.

In [ ]:
def plot_comparison(base_f1, skopt_f1, hyperopt_f1, base_time, skopt_time, hyperopt_time):
    """
    Genera gráficos comparativos de F1-Score y tiempos de ejecución.
    
    Parámetros:
    base_f1: F1-Score modelo base
    skopt_f1: F1-Score BayesSearchCV
    hyperopt_f1: F1-Score Hyperopt
    base_time: Tiempo de entrenamiento modelo base
    skopt_time: Tiempo de BayesSearchCV
    hyperopt_time: Tiempo de Hyperopt
    """
    # Gráfico de F1-Scores
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    models = ['Base', 'BayesSearchCV', 'Hyperopt']
    f1_scores = [base_f1, skopt_f1, hyperopt_f1]
    colors = ['skyblue', 'lightgreen', 'salmon']
    bars = plt.bar(models, f1_scores, color=colors)
    plt.title('Comparación de F1-Score', fontsize=14)
    plt.ylabel('F1-Score', fontsize=12)
    plt.ylim(0.9, 1.0)
    
    # Añadir valores en las barras
    for bar in bars:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=10)
    
    # Gráfico de tiempos
    plt.subplot(1, 2, 2)
    times = [base_time, skopt_time, hyperopt_time]
    colors_time = ['skyblue', 'lightgreen', 'salmon']
    bars_time = plt.bar(models, times, color=colors_time)
    plt.title('Tiempos de Ejecución', fontsize=14)
    plt.ylabel('Segundos', fontsize=12)
    
    # Añadir valores en las barras
    for bar in bars_time:
        height = bar.get_height()
        plt.annotate(f'{height:.2f}s',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=10)
    
    plt.tight_layout()
    plt.savefig('comparacion_resultados.png', dpi=300)
    plt.show()

def create_main_results_table(base_report, base_f1, base_time,
                             skopt_params, skopt_report, skopt_f1, skopt_time,
                             hyperopt_params, hyperopt_report, hyperopt_f1, hyperopt_time):
    """
    Crea DataFrames con los resultados principales de los tres modelos.
    
    Retorna:
    tuple: (df_base, df_skopt, df_hyperopt, df_comparacion)
    """
    # DataFrame para modelo base
    df_base = pd.DataFrame({
        'Modelo': ['Base'],
        'F1-Score': [base_f1],
        'Tiempo (s)': [base_time],
        'Reporte': [base_report]
    })
    
    # DataFrame para BayesSearchCV
    df_skopt = pd.DataFrame({
        'Modelo': ['BayesSearchCV'],
        'F1-Score': [skopt_f1],
        'Tiempo (s)': [skopt_time],
        'Hiperparámetros': [str(skopt_params)],
        'Reporte': [skopt_report]
    })
    
    # DataFrame para Hyperopt
    df_hyperopt = pd.DataFrame({
        'Modelo': ['Hyperopt'],
        'F1-Score': [hyperopt_f1],
        'Tiempo (s)': [hyperopt_time],
        'Hiperparámetros': [str(hyperopt_params)],
        'Reporte': [hyperopt_report]
    })
    
    # DataFrame comparativo final
    df_comparacion = pd.DataFrame({
        'Modelo': ['Base', 'BayesSearchCV', 'Hyperopt'],
        'F1-Score': [base_f1, skopt_f1, hyperopt_f1],
        'Tiempo (s)': [base_time, skopt_time, hyperopt_time],
        'Mejora vs Base': ['-', 
                          f"{(skopt_f1 - base_f1):.4f}", 
                          f"{(hyperopt_f1 - base_f1):.4f}"]
    })
    
    return df_base, df_skopt, df_hyperopt, df_comparacion

def display_formatted_tables(*tables):
    """
    Muestra las tablas de resultados formateadas en la consola.
    
    Parámetros:
    tables: Tupla de DataFrames a mostrar
    """
    titles = [
        "RESULTADOS MODELO BASE",
        "RESULTADOS BAYESSEARCHCV",
        "RESULTADOS HYPEROPT",
        "COMPARACIÓN FINAL DE MODELOS"
    ]
    
    for i, df in enumerate(tables):
        print(titles[i])
        print("="*70)
        
        # Mostrar reportes completos solo para los modelos principales
        if i < 3:           
            # Mostrar el DataFrame sin la columna de reporte
            df_to_show = df.drop(columns=['Reporte'])
            if 'Hiperparámetros' in df_to_show.columns:
                # Formatear hiperparámetros para mejor visualización
                df_to_show['Hiperparámetros'] = df_to_show['Hiperparámetros'].apply(
                    lambda x: '\n' + str(x).replace(',', '\n').replace('{', '').replace('}', ''))
            
            print("\nMétricas Resumen:")
            print(df_to_show.to_string(index=False, justify='left'))
        else:
            print(df.to_string(index=False, justify='left'))
        
        print("\n")

**Bloque 6:** Función de ejecución.

- **`main()`**
Función principal que ejecuta todo el flujo: carga de datos, entrenamiento, optimización, experimentos, visualizaciones y resumen final.

In [ ]:
def main():
    """Función principal que ejecuta todo el flujo de trabajo"""
    # 1. Carga y preparación de datos
    print("Cargando y preparando datos...")
    data = load_breast_cancer()
    X_train, X_test, y_train, y_test = load_and_prepare_data(data)
    
    # 2. Modelo base
    print("\nEntrenando modelo base...")
    base_report, base_f1, base_time = train_base_rf(X_train, y_train, X_test, y_test)
    
    # 3. Optimización con BayesSearchCV
    print("\nOptimizando con BayesSearchCV (Scikit-Optimize)...")
    skopt_params, skopt_report, skopt_f1, skopt_time = bayes_opt_skopt(X_train, y_train, X_test, y_test)
    
    # 4. Optimización con Hyperopt
    print("\nOptimizando con Hyperopt...")
    hyperopt_params, hyperopt_report, hyperopt_f1, hyperopt_time = bayes_opt_hyperopt(X_train, y_train, X_test, y_test)
    
    # 5. Crear DataFrames principales
    df_base, df_skopt, df_hyperopt, df_comparacion = create_main_results_table(
        base_report, base_f1, base_time,
        skopt_params, skopt_report, skopt_f1, skopt_time,
        hyperopt_params, hyperopt_report, hyperopt_f1, hyperopt_time
    )
    
    # 6. Mostrar todas las tablas
    display_formatted_tables(
        df_base, df_skopt, df_hyperopt, df_comparacion
    )
    
    # 7. Gráfico comparativo básico
    plot_comparison(base_f1, skopt_f1, hyperopt_f1, base_time, skopt_time, hyperopt_time)
    print("Gráfico comparativo guardado como 'comparacion_resultados.png'")

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Conclusiones, recomendaciones y reflexiones finales

## Conclusiones

- A partir de los resultados obtenidos, se concluye que la **Optimización Bayesiana** (tanto con **Scikit-Optimize** como con **Hyperopt**) puede superar al modelo base, pero solo bajo ciertas condiciones adecuadas de configuración.
- El modelo base, sin ajuste de hiperparámetros, ya presentaba un desempeño sólido, con un **F1-Score de aproximadamente 0.9488**.
- En los primeros experimentos con pocas iteraciones (por ejemplo, 20) y espacios de búsqueda estrechos, los métodos de optimización no lograron mejorar consistentemente al modelo base. De hecho, en algunos casos el rendimiento fue menor.
- Este hallazgo sugiere que la optimización bayesiana **no garantiza automáticamente** una mejora significativa, especialmente cuando se limita el número de evaluaciones o el rango de búsqueda de hiperparámetros.

## Recomendaciones y reflexiones finales

Durante el desarrollo de este trabajo se evidenció que el uso de técnicas avanzadas como la optimización bayesiana requiere no solo una buena comprensión de los algoritmos involucrados, sino también un **criterio técnico claro** para determinar cuándo vale la pena aplicarlas.

### Recomendaciones clave

- **Evaluar primero un modelo base bien construido.**  
  En muchos casos, un modelo con hiperparámetros por defecto (como `RandomForestClassifier`) ya puede ofrecer resultados competitivos, especialmente cuando se dispone de poco tiempo o recursos computacionales limitados.
- **Aplicar optimización solo cuando se justifique.**  
  Si el modelo base tiene margen de mejora o el caso de uso exige un desempeño elevado (por ejemplo, en aplicaciones clínicas o de alto impacto), entonces sí se justifica el uso de técnicas más complejas como `BayesSearchCV` o `Hyperopt`.
- **Ajustar correctamente las configuraciones.**  
  El número de iteraciones y el tamaño del espacio de búsqueda son factores críticos. Pocas iteraciones o rangos muy limitados pueden llevar a un desempeño peor que el del modelo base.
- **Considerar el costo computacional.**  
  Mientras que el modelo base se entrenó en aproximadamente **0.13 segundos**, las versiones optimizadas requirieron entre **20 y 26 segundos**. En contextos donde el tiempo o los recursos importan (como en producción o en grandes volúmenes de datos), esto podría ser un factor relevante.
- **No confundir complejidad con efectividad.**  
  Añadir múltiples capas de optimización no garantiza mejores resultados. La clave está en una **planificación estratégica** del experimento y una evaluación cuidadosa del valor agregado por cada técnica.